In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from functools import reduce
import os

In [2]:
# -----------------------------
# 1. Cargar datos desde CSV
# -----------------------------
tesis_df = pd.read_csv("csv_ranking_2/tesis.csv")
#Crea una columna llamada investigador con el nombre y el apellido
tesis_df['investigador'] = tesis_df['Nombre'].str.strip() + ' ' + tesis_df['Apellidos'].str.strip()

pubs_df = pd.read_csv("csv_ranking_2/indicadores.csv")
pubs_df['investigador'] = pubs_df['Nombre'].str.strip() + ' ' + pubs_df['Apellidos'].str.strip()
pubs_df = pubs_df.rename(columns={
    'Q1 Totales': 'q1_totales',
    'Publicaciones Totales': 'pubs_total'
})

proy_df = pd.read_csv("csv_ranking_2/proyectos.csv")
colab_df = pd.read_csv("csv_ranking_2/colaboraciones_ICC.csv", sep=';', decimal=',')

# -----------------------------
# 2. Agregar métricas por investigador
# -----------------------------
#No se están cogiendo bien los datos de docencia
#tesis_agg = tesis_df.groupby('investigador').size().reset_index(name='tesis_total')
#tesis_agg = tesis_df[['investigador', '4']].rename(columns={'4': 'tesis_total'})
tesis_agg = tesis_df[['investigador', 'Tesis Dirigidas']]

# Calcular puntaje investigación con la fórmula dada:
# (Publicaciones Totales - Q1 Totales)*1 + Q1 Totales*3
pubs_df['investigacion_puntaje'] = (pubs_df['pubs_total'] - pubs_df['q1_totales']) * 1 + pubs_df['q1_totales'] * 3

#((Publicaciones totales - Q1 Totales)*0,25 + Q1 totales*0,75)/(Publicaciones Totales)
#pubs_df['investigacion_puntaje'] = ((pubs_df['pubs_total'] - pubs_df['q1_totales']) * 0.25 + pubs_df['q1_totales'] * 0.75)/(pubs_df['pubs_total'])

# Agrupar por investigador sumando el puntaje de investigación
pubs_invest_agg = pubs_df.groupby('investigador')['investigacion_puntaje'].sum().reset_index()

#p83 = proy_df[proy_df['categoria']==83]
p83 = proy_df[proy_df['categoria'].str.contains("CONTRATOS ART. 60", case=False)]
p83_num = p83.groupby('investigador').size().reset_index(name='p83_num')
p83_euros = p83.groupby('investigador')['euros'].sum().reset_index(name='p83_euros')

resp = proy_df[proy_df['responsable']==True]
resp_num = resp.groupby('investigador').size().reset_index(name='resp_num')
resp_euros = resp.groupby('investigador')['euros'].sum().reset_index(name='resp_euros')

#colab_agg = colab_df.groupby('investigador').size().reset_indexname='colab_total')
colab_agg = colab_df[['investigador','ICC Normalizado']]

dfs = [tesis_agg, pubs_invest_agg, p83_num, p83_euros, resp_num, resp_euros, colab_agg]
master_df = reduce(lambda left, right: left.merge(right, on='investigador', how='outer'), dfs).fillna(0)

# -----------------------------
# 3. Normalización
# -----------------------------
metric_cols = ['Tesis Dirigidas', 'investigacion_puntaje', 'p83_num', 'p83_euros', 'resp_num', 'resp_euros', 'ICC Normalizado']
data = torch.tensor(master_df[metric_cols].values, dtype=torch.float32)
#Saca valores minimos y maximos para normalizar
mins = data.min(dim=0, keepdim=True).values
maxs = data.max(dim=0, keepdim=True).values
#Normaliza los valores
normed = (data - mins) / (maxs - mins + 1e-6)

# -----------------------------
# 4. Cálculo de scores
# -----------------------------
#Coge las métricas necesarias para cada caso
idx = {
    'docencia':      [0],
    'investigacion': [1], 
    'transferencia': [2,3],
    'gestion':       [4,5,6],
}

cat_scores = []
for cat, cols in idx.items():
    score = normed[:, cols].mean(dim=1, keepdim=True)
    cat_scores.append(score)
    master_df[f'{cat}_score'] = score.squeeze().numpy()

all_scores = torch.cat(cat_scores, dim=1).mean(dim=1)
master_df['score_total'] = all_scores.numpy()

#Ranking con pesos
#weights = torch.tensor([0.0833, 0.0833, 0.0833, 0.75])  # Docencia, Investigación, Transferencia, Gestión
#cat_scores_tensor = torch.cat(cat_scores, dim=1)       # shape: (n_investigadores, 4)
#weighted_score = (cat_scores_tensor * weights).sum(dim=1)
#master_df['score_total'] = weighted_score.numpy()

ranking_df = master_df.sort_values('score_total', ascending=False)

# -----------------------------
# 5. Radar chart por investigador
# -----------------------------
def make_radar_chart(row, output_dir):
    categories = ['Docencia', 'Investigación', 'Transferencia', 'Gestión']
    values = [
        row['docencia_score'],
        row['investigacion_score'],
        row['transferencia_score'],
        row['gestion_score']
    ]
    values += values[:1]  # cerrar el círculo

    angles = [n / float(len(categories)) * 2 * pi for n in range(len(categories))]
    angles += angles[:1]

    plt.figure(figsize=(4, 4))
    ax = plt.subplot(111, polar=True)
    ax.plot(angles, values, linewidth=2)
    ax.fill(angles, values, alpha=0.4)
    #Esto es para que haya una escala fija y se diferencien mejor los gráficos
    ax.set_ylim(0, 1)
    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=7)
    
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.set_title(row['investigador'], size=10, pad=10)
    plt.tight_layout()
    
    filepath = os.path.join(output_dir, f"{row['investigador']}.png")
    plt.savefig(filepath)
    plt.close()
    return filepath

# -----------------------------
# 6. Guardar en Excel con gráficos
# -----------------------------
output_excel = "ranking_con_graficos_2.xlsx"
chart_dir = "graficos_temp"
os.makedirs(chart_dir, exist_ok=True)

master_df['grafico_path'] = master_df.apply(lambda row: make_radar_chart(row, chart_dir), axis=1)

with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
    master_df.to_excel(writer, sheet_name='Ranking', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Ranking']

    #Incluir graficos en el excel
    #for idx, row in master_df.iterrows():
    #    img_path = row['grafico_path']
    #   worksheet.insert_image(f'L{idx+2}', img_path, {'x_scale': 0.5, 'y_scale': 0.5})

print(f"Ranking exportado a: {output_excel}")

Ranking exportado a: ranking_con_graficos_2.xlsx
